In [ ]:
!pip install librosa

In [81]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import shutil
import librosa
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

In [63]:
# Folder Locations

dataset = "./open/"
TRAIN_WAV = dataset + "train/"
TEST_WAV = dataset + "test/"
PREPROCESSED = dataset + "preprocessed_data/"
TRAIN_LABEL_SEP = PREPROCESSED + "train_label_sep/"
WAV_TRAIN_LABEL_SEP = PREPROCESSED + "wav_train_label_sep/"


if not os.path.exists(dataset + "preprocessed_data"):
    os.mkdir(dataset + "preprocessed_data")
    
if not os.path.exists(PREPROCESSED + "train_label_sep"):
    os.mkdir(PREPROCESSED + "train_label_sep")
    
if not os.path.exists(PREPROCESSED + "wav_train_label_sep"):
    os.mkdir(PREPROCESSED + "wav_train_label_sep")

In [66]:
wav_file_dict = {"train_wav" : TRAIN_WAV,
                "test_wav" : TEST_WAV,
                 "wav_sep" : WAV_TRAIN_LABEL_SEP
                 }

wav_file_locations = {}
for key, value in wav_file_dict.items():
    wav_file_locations[key] = glob.glob(value + "*.wav")
    
csv_file_dict = {"train_label_sep" : TRAIN_LABEL_SEP,
                 "wav_train_label_sep" : WAV_TRAIN_LABEL_SEP
                }

csv_file_location = {}
for key, value in csv_file_dict.items():
    csv_file_location[key] = glob.glob(value + "*.csv")

In [4]:
# Local definitions - 1

def clean_name(column_name):
    '''
    column name 중 변경할 사항이 있다면 아래 코드를 변경해주세요, 
    ~ 로 스플릿한 구절 중 0번째 len이 7인 경우; 예시: "(220123"
    ) 이후의 course name을 return합니다.
    
    아래와 같이 사용해 주시면 편합니다.
    df["course_name"] = df["course_name"].apply(clean_course_name)

    '''
    if column_name.startswith("."):
        # Classify the date range to cleanse.
        temp = column_name.replace(".","")
        temp = temp.replace("wav", ".wav")
        column_name = "./open" + temp
        return column_name
    else:
        return column_name

In [82]:
# Transformer

class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads, num_layers, hidden_dim, dropout):
        super(TransformerModel, self).__init__()
        
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.pos_encoding = PositionalEncoding(hidden_dim)
        
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_dim, num_heads, dim_feedforward=hidden_dim, dropout=dropout),
            num_layers
        )
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        
        x = x.permute(1, 0, 2)  # Reshape for transformer input
        x = self.transformer_encoder(x)
        
        x = x.permute(1, 0, 2)  # Reshape back to (batch_size, seq_len, hidden_dim)
        x = torch.mean(x, dim=1)  # Average pooling over sequence length
        
        x = self.fc(x)
        return F.softmax(x, dim=1)

# positional encoding

class PositionalEncoding(nn.Module):
    def __init__(self, hidden_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, hidden_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_dim, 2).float() * (-math.log(10000.0) / hidden_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

In [5]:
train_csv = pd.read_csv(dataset + "train.csv")
train_csv

,id,path,label
0,TRAIN_0000,./train/TRAIN_0000.wav,1
1,TRAIN_0001,./train/TRAIN_0001.wav,2
2,TRAIN_0002,./train/TRAIN_0002.wav,4
3,TRAIN_0003,./train/TRAIN_0003.wav,5
4,TRAIN_0004,./train/TRAIN_0004.wav,4
...,...,...,...
4996,TRAIN_4996,./train/TRAIN_4996.wav,5
4997,TRAIN_4997,./train/TRAIN_4997.wav,0
4998,TRAIN_4998,./train/TRAIN_4998.wav,1
4999,TRAIN_4999,./train/TRAIN_4999.wav,1


In [6]:
for label in train_csv['label'].unique():
    filtered_csv = train_csv[train_csv['label'] == label]
    filename = f"label_{label}.csv"
    filtered_csv.to_csv(TRAIN_LABEL_SEP + filename, index=False)

In [7]:
for file in csv_file_location["train_label_sep"]:
    csv_file = pd.read_csv(file)
    csv_file['path'] = csv_file['path'].apply(clean_name)
    filename = os.path.basename(file)
    csv_file.to_csv(TRAIN_LABEL_SEP + filename, index=False)
    

In [62]:
parent_folder = TRAIN_WAV



for label in tqdm(train_csv['label'].unique(), desc='Processing folders'):
    features_df = pd.DataFrame()
    filtered_csv_filename = f"label_{label}.csv"
    filtered_csv = pd.read_csv(os.path.join(TRAIN_LABEL_SEP, filtered_csv_filename))
    
    target_folder = os.path.join(WAV_TRAIN_LABEL_SEP, f"label_{label}")
    
    for path in filtered_csv['path']:
        wav_filename = os.path.basename(path)
        target_path = os.path.join(parent_folder, wav_filename)
        audio, sr = librosa.load(target_path)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr)
        mfcc_flattened = mfcc.flatten()
        row = pd.DataFrame([list(mfcc_flattened) + [label]])
        index_name = wav_filename.split(".")[0]
        row.index = [index_name]
        features_df = pd.concat([features_df, row])
    
    

    features_df.sort_index(inplace=True)
    filename = WAV_TRAIN_LABEL_SEP + f"label_{label}.csv"
    features_df.to_csv(filename)

Processing files:  27%|██████▎                | 232/848 [00:12<00:34, 17.87it/s]


Processing files:  54%|████████████▍          | 459/848 [00:25<00:24, 15.57it/s]


Processing files:  82%|██████████████████▊    | 694/848 [00:39<00:10, 14.72it/s]


Processing files:   6%|█▍                      | 50/859 [00:06<02:25,  5.56it/s]


Processing files:  36%|████████▏              | 306/859 [00:20<00:26, 21.20it/s]


...

Processing files:  14%|███▏                   | 118/867 [00:06<00:30, 24.23it/s]


Processing files:  41%|█████████▍             | 357/867 [00:19<00:29, 17.10it/s]


Processing files:  67%|███████████████▍       | 580/867 [00:32<00:15, 18.20it/s]


Processing files:  91%|█████████████████████  | 792/867 [00:44<00:05, 14.54it/s]


Processing folders: 100%|█████████████████████████| 6/6 [04:56<00:00, 49.40s/it]


In [68]:
for file in csv_file_location["wav_train_label_sep"]:
    df = pd.read_csv(file)
    df.info()
    print("*"*10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853 entries, 0 to 852
Columns: 3642 entries, Unnamed: 0 to 3640
dtypes: float64(3641), object(1)
memory usage: 23.7+ MB
**********
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722 entries, 0 to 721
Columns: 4122 entries, Unnamed: 0 to 4120
dtypes: float64(4121), object(1)
memory usage: 22.7+ MB
**********
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867 entries, 0 to 866
Columns: 4142 entries, Unnamed: 0 to 4140
dtypes: float64(4141), object(1)
memory usage: 27.4+ MB
**********
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848 entries, 0 to 847
Columns: 3642 entries, Unnamed: 0 to 3640
dtypes: float64(3641), object(1)
memory usage: 23.6+ MB
**********
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Columns: 4202 entries, Unnamed: 0 to 4200
dtypes: float64(4201), object(1)
memory usage: 27.3+ MB
**********
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Columns: 3862 entries, 

In [70]:
for file in csv_file_location["wav_train_label_sep"]:
    df = pd.read_csv(file)
    print(df.describe())
    print("*"*10)

                0           1           2           3           4           5  \
count  853.000000  853.000000  853.000000  853.000000  853.000000  853.000000   
mean  -492.641885 -469.683573 -467.581940 -465.355790 -463.711685 -462.090440   
std     47.483778   40.417223   43.728951   47.223434   48.786415   48.913684   
min   -731.100040 -642.316700 -570.678160 -560.757100 -563.332500 -559.079700   
25%   -508.098000 -482.936370 -483.201400 -481.955630 -480.736360 -480.477630   
50%   -492.639620 -470.837250 -471.451700 -470.530520 -469.624100 -469.385200   
75%   -472.993320 -458.383330 -458.896580 -457.081820 -456.623080 -455.693730   
max   -230.032090 -135.379170  -91.832430  -32.339530  -29.210724  -62.826134   

                6           7           8           9  ...      3631  \
count  853.000000  853.000000  853.000000  853.000000  ...  1.000000   
mean  -458.833929 -454.149611 -449.398272 -444.118554  ... -6.775582   
std     50.307003   55.037121   61.370094   67.454497 

                0           1           2           3           4           5  \
count  852.000000  852.000000  852.000000  852.000000  852.000000  852.000000   
mean  -500.826520 -479.538672 -478.830243 -477.662818 -476.580220 -475.342869   
std     40.924066   31.727651   33.386990   36.623097   39.298850   41.721973   
min   -786.700800 -658.666300 -658.760560 -659.239440 -659.829960 -659.339970   
25%   -514.079325 -488.708192 -490.094605 -489.146180 -489.395927 -488.491400   
50%   -500.363755 -478.745265 -479.772560 -479.377445 -478.932950 -478.354115   
75%   -482.308585 -467.319480 -467.629720 -467.029238 -466.424537 -467.051587   
max   -278.111420 -253.310120 -250.626190 -150.022190 -124.010315 -117.972760   

                6           7           8           9  ...      4191  \
count  852.000000  852.000000  852.000000  852.000000  ...  2.000000   
mean  -473.791077 -472.219821 -470.373893 -468.375570  ... -5.651120   
std     44.038979   45.972814   47.889833   49.394085 